<a href="https://colab.research.google.com/github/mattiadutto/aml_federeted_learning/blob/main/aml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# From: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net = net.to("cuda")
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
import torch.optim as optim

E = 2

class Client():
  def __init__(self, train_set, n):
    self.train_set = train_set
    self.n = n
    self.batch_size = 32
    self.train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    self.net = Net()
    self.net = self.net.to("cuda")
    # create your optimizer
    self.optimizer = optim.SGD(net.parameters(), lr=0.01)
    self.criterion = nn.CrossEntropyLoss()
    
  def clientUpdate(self, parameters):
    net.load_state_dict(parameters)
    theta = parameters
    for e in range(E):
      for images, labels in self.train_loader:
        images = images.to("cuda")
        labels = labels.to("cuda")
        # in your training loop:
        self.optimizer.zero_grad()   # zero the gradient buffers
        output = net(images)
        loss = self.criterion(output, labels)
        loss.backward()
        self.optimizer.step()    # Does the update
    
    return_dict = {}
    for (k1, v1), (k2, v2) in zip(parameters.items(), net.state_dict().items()):
      return_dict[k1] = v1 - lr * v2
    return return_dict

In [10]:
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random

# K = 1, NUMBE_OR_CLIENTS = 2, MAX_TIME = 3 -> 58 sec

K = 1 # to set
NUMBER_OF_CLIENTS = 2 # to set
MAX_TIME = 3 #to set

batch_size = 32

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

trainset_len = ( len(trainset) // NUMBER_OF_CLIENTS ) * NUMBER_OF_CLIENTS
print(trainset_len)
trainset = torch.utils.data.Subset(trainset, list(range(trainset_len)))

lengths = len(trainset) // NUMBER_OF_CLIENTS * np.ones(NUMBER_OF_CLIENTS, dtype=np.int)
print(lengths)
trainsets = torch.utils.data.random_split(dataset=trainset, lengths=lengths)

lr = 0.01 

clientsSizes = torch.zeros(NUMBER_OF_CLIENTS)
clients = list()

def selectClients(k):
  return random.sample(clients, k)

def aggregateClient(deltaThetas):
  parameters = None
  for i,d in enumerate(deltaThetas):
    if i == 0:
      parameters = d
    else:
      for (k, v) in d.items():
        parameters[k] += len(trainsets[i])/len(trainset) * v
   
  return parameters

for c in range(NUMBER_OF_CLIENTS):
  clients.append(Client(trainsets[c], len(trainset)))

for t in range(MAX_TIME):
  clients = selectClients(K)
  deltaThetas = list()
  for i, c in enumerate(clients):
    deltaThetas.append(c.clientUpdate(net.state_dict()))
  g = aggregateClient(deltaThetas)
  
  parameters = {}
  for (k1, v1), (k2, v2) in zip(net.state_dict().items(), g.items()):
    parameters[k1] = v1 - lr * v2
  net.load_state_dict(parameters)

Files already downloaded and verified
Files already downloaded and verified
50000
[25000 25000]
